# AI SQL
AISQL delivers an easy-to-use composable query language that handles both text and multimodal data through familiar SQL syntax. Our high-performance batch engine processes queries faster through intelligent query optimization, while delivering lower costs than traditional AI solutions. Natively ingest structured and unstructured data into unified multimodal tables, enabling comprehensive insights through familiar SQL analytics for all your data.

## AI SQL Benefits:
### 1) Easy-to-use SQL syntax to build AI pipelines without complex coding
### 2) High-performance processing across all modalities (text, image, audio)
### 3) Lower cost batch processing through optimized architecture to support faster and larger batch jobs.


## Three Major function calls
### AI_FILTER
    AI-powered SQL operator for semantic filtering. You can use the same syntax on a single table (for filtering) or join multiple tables together upon a semantic relationship. 

### AI_AGG/AI_SUMMARIZE_AGG
    Advanced functionalities provide unmatched comprehensive analysis across multiple data entries with a user prompt. This method overcomes contextual window constraints, facilitating extensive multi-record examination with no comparable solutions in the market.

### AI_CLASSIFY
    Functions for easy classification across multi-modal data

# Resturant Review Batch Unstructured Processing

Snowflake's AI powered functions enable column level operations with LLM like traditional database operators. 

Image Processing
1. Load data into FILE column
2. Using COMPLETE(IMAGE) Function to add descriptions for each image
3. Using AI_FILTER(IMAGE) and AI_CLASSIFY(IMAGE) to filter/classify accordingly.
4. Using AI_FILTER() to join food IMAGE with corresponding text from the food menu.

With this import we can run streamlit, pandas, and the snowbook extras python script we imported.  The snowbook extras allows us to view the images in table format below.

In [ ]:
import streamlit as st
import pandas as pd
import snowbooks_extras

In [ ]:
create or replace table img_tbl as
select to_file(file_url) img_file, * from directory(@FOODIMAGES);

In [ ]:
desc table img_tbl;

Below we can see the file images using our snowbooks_extras.py helper file and see the data we are working with.  You can click the image below to see a larger image

In [ ]:
select img_file, file_url, last_modified from img_tbl

Below we use a helper function in the cortex complete to prompt to read the image and write an advertisement for the dish in the image.

In [ ]:
-- test complete(image)
select 
    img_file,
    snowflake.cortex.complete('pixtral-large', prompt('Write me an advertisement tagline for this dish: {0}', img_file)),
    file_url,
    last_modified
    from img_tbl;

Now we will run a new [AI_FILTER](https://docs.snowflake.com/LIMITEDACCESS/snowflake-cortex/ai_filter) to filter out the images and look only for those that are desserts from our list of images. We want to user AI_FILTER with a prompt to help us tell if the image in the picture is a dessert.  Please edit the code below in the where statement properly use the AI_FILTER

In [ ]:
select img_file, relative_path, file_url, last_modified from img_tbl
--Add a where clause that uses AI_FILTER and has a prompt trying to find out if the image is a dessert

Let's further classify these images using [AI_CLASSIFY](https://docs.snowflake.com/LIMITEDACCESS/snowflake-cortex/ai_classify).  We specify what classifications we want in the prompt and ask the GenAI model to break up the images into those classes

In [ ]:
select
    img_file, 
    relative_path,
    --add an AI_CLASSIFY function with a prompt that will help you classify the food into a dessert, drink, main dish, or side dish
    ,
    file_url, last_modified
from img_tbl;

We are now going to create a menu, we will simply do this by using a regex expression to read the file name and remove special characters and numbers as our files have pretty good descriptors of the contents.  We store this into the FOOD_IMAGE for later processing

In [ ]:
create or replace table food_menu as
select relative_path, REPLACE(
        REGEXP_REPLACE(
            REGEXP_REPLACE(relative_path, '^class_', ''),
            '_[0-9]+\\.jpg$', ''
        ),
        '_', ' '
    ) as item
from img_tbl;

select * from food_menu;

Lets use AI_FILTER to do a join from the food_menu to the img_tble.  Here we are joining text to an image.  We can see the restuls are pretty good but not perfect.

CROSS JOIN + AI_FILTER can also be used to do semantic joins, leveraging AI capacity to match between two tables.


In [ ]:
select img_tbl.img_file,
    food_menu.item
    from food_menu
    join img_tbl
    -- join on an AI_FILTER with a prompt that the image is referring to what the item description is that we just created

Lets take a look back at just pure text and AI functions there. We will look at the DOORDASH_100 table which has multiple rows of reviews and we will use AI_AGG to aggregate those together

In [ ]:
select * from doordash_100;

Let's use cortex complete to create structured insights into our reviews from above.  We want to pull out primary issues, delivery timeliness, food temperature, value assessment, and sepcial occasion from our reviews from DoorDash

In [ ]:
SELECT *,

       -- snowflake.cortex.sentiment(review_text) AS sentiment_score,

       snowflake.cortex.complete('claude-3-5-sonnet', 
       [{
        'role': 'user',
        'content': CONCAT(

               'Analyze the following review text and extract key details. Respond ONLY with a JSON object containing the following keys:

               1. "primary_issue": Categorize the main issue into one of the following: ["food_quality", "delivery_experience", "order_accuracy", "customer_service", "no_issue"]. If the sentiment is positive, use "no_issue".

               2. "delivery_timeliness": If applicable, categorize as "delayed", "on_time", "early", or "not_mentioned".

               3. "food_temperature": Categorize as "too_cold", "too_hot", "just_right", or "not_mentioned".

               4. "value_assessment": Categorize as "overpriced", "worth_it", "great_value", or "not_mentioned".

               5. "special_occasion": Indicate if this was for a special occasion (like a birthday or anniversary) as "yes" or "no".

               DO NOT MAKE ANYTHING UP - FOLLOW THE ABOVE CATEGORIES

               Ensure that the JSON object is complete and that each value is concise.

               Example response:

               {"primary_issue": "food_quality", "delivery_timeliness": "delayed", "food_temperature": "too_cold", "value_assessment": "overpriced", "special_occasion": "yes", "would_order_again": "no"}

               Provide ONLY the JSON object in your response, with no additional text.

               ', review_text)
        }],
        {
        'response_format':{
            'type':'json',
            'schema':{'type' : 'object','properties': {
                'primary_issue': {'type': 'string'},
                'delivery_timeliness': {'type': 'string'},
                'food_temperature': {'type': 'string'},
                'value_assessment': {'type': 'string'},
                'special_occasion': {'type': 'string'}
            }
            }}
        }
       ) AS review_response,
    review_response:structured_output[0]:raw_message as parsed_review,
    PARSE_JSON(parsed_review):primary_issue::STRING AS primary_issue,
    PARSE_JSON(parsed_review):delivery_timeliness::STRING AS delivery_timeliness,
    PARSE_JSON(parsed_review):food_temperature::STRING AS food_temperature,
    PARSE_JSON(parsed_review):value_assessment::STRING AS value_assessment,
    PARSE_JSON(parsed_review):special_occasion::STRING AS special_occasion

FROM doordash_100;

Now we can filter those reviews using [AI_FILTER (PrPr)](https://docs.snowflake.com/LIMITEDACCESS/snowflake-cortex/ai_filter-snowflake-cortex) on the context we added previously and look for reviews that mentioned a special occasion like a birthday or other

In [ ]:
-- Find reviews mentioning special occasions
SELECT 
    restaurant_name,
    RESTAURANT_CUISINE_TYPE,
    order_type,
    rating,
    review_text
FROM doordash_100
--add a where clause with AI_FILTER to look for review text that mentiones a birthday or special occasion

Now we can also apply an aggregate function using [AI_AGG](https://docs.snowflake.com/LIMITEDACCESS/snowflake-cortex/ai_agg).  We will bring together reviews that have more than one for the same resturant and see what the system comes back with.  Most of our data has only one review for a resturant but there are 3 that have 2 so we can focus on those.

There is nothing to change in this code to get it to run, since the prompt was a little complex we decided to just show you how you could use this feature

In [ ]:
-- 4.1 Restaurant Review Analysis By Cusine type.
-- Extract key insights and improvement recommendations for each restaurant
create or replace table aiagg_Strengths as (
SELECT  RESTAURANT_NAME,
        RESTAURANT_CUISINE_TYPE,
        AI_AGG('Resturant Name: '|| RESTAURANT_NAME || '\n\nCuisine type: '|| RESTAURANT_CUISINE_TYPE || '\n\nFeedback: ' || review_text, 
       'Analyze these reviews for this specific cuisine type of restaurant. Identify the top strengths and weaknesses mentioned by customers. Provide 2-3 specific recommendations for improvement based on customer feedback.') as analysis
FROM doordash_100 
where restaurant_id in (749,920,628)
GROUP BY all);

select * from aiagg_Strengths

Now lets aggregate to a delivery type level.  Now we are bringing together more rows into just 3 output levels of delivery, dine-in, and takeout

In [ ]:
-- 5.1 Feedback Summarization by Order Type
create or replace table aiagg_OrderType as (
SELECT
    order_type,
    AI_AGG('Order type: '|| order_type || '\n\nFeedback: ' || review_text, 
        'Analyze this customer feedback for DoorDash operations. Identify the main pain points and positive aspects of the experience. What specific operational improvements could be made?') 
        AS operational_insights
FROM doordash_100
GROUP BY order_type);

select * from aiagg_OrderType;

Finally we will summarize across all rating levels to see what people are saying using the [SUMMARIZE_AGG](https://docs.snowflake.com/LIMITEDACCESS/snowflake-cortex/summarize_agg) function.

In [ ]:
-- 5.2 Restaurant Rating Analysis
-- Generate summaries by rating category to understand different customer experiences
create or replace table summarizeagg_experience as (
SELECT
    rating,
    --add SUMMARIZE_AGG function that will summarize the review_text and create a new column name experience_summary
FROM doordash_100
GROUP BY rating
order by rating);

Select * from summarizeagg_experience;

Lets use [AI_CLASSIFY](https://docs.snowflake.com/LIMITEDACCESS/snowflake-cortex/ai_classify) to classify the column into categories (food quality issues, portion sizes and pricing, delivery time and effeiceincy, ambiance and atmosphere, customer service, flavor)

In [ ]:
-- 4.1 Restaurant Review Analysis By Cusine type.
-- Extract key insights and improvement recommendations for each restaurant
create or replace table aiclassify_review as (
SELECT  rating,
review_date,
order_type,
review_text,
Restaurant_name,
Restaurant_cuisine_type,
--use the AI_CLASSIFY function to analyze review_text and classify the following types of reviews: food quality issues, portion sizes and pricing, delivery time and efficiency, ambiance and atmosphere, customer service, flavor and name it the analysis column
FROM doordash_100
GROUP BY all);

select * from aiclassify_review;